# Process data config

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import json
import joblib
from collections import defaultdict
from tqdm.auto import tqdm

import lightgbm as lgb

from eli5.sklearn import PermutationImportance
from shaphypetune import BoostBoruta

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import log_loss, mean_squared_error, precision_score

import torch

from colorama import Style, Fore


class CFG:
    select_features = False
    train_NN = False
    train_LGBM = True
    n_repeats = 1
    n_folds = 5

# Load data and add indicators

In [2]:
# last previous data point to collect for model training (value represents number of hours before signal point)
last = 96

df = pd.read_pickle(f'signal_stat/train_df_{last}.pkl')

df = df[df['pattern'].isin(['MACD', 'STOCH_RSI'])]

display(df.head())
display(df.shape)


,time,open,high,low,close,volume,rsi,stoch_slowk,stoch_slowd,stoch_slowk_dir,...,linear_reg_angle_prev_96,macd_prev_96,macdsignal_prev_96,macdhist_prev_96,macd_dir_prev_96,macdsignal_dir_prev_96,atr_prev_96,close_smooth_prev_96,target,ttype
2,2022-10-18 23:00:00,23.4600,23.7000,23.4400,23.6200,3346.24,43.727673,25.327885,24.045347,-0.007431,...,-6.562901,0.035983,0.107527,-0.071544,-0.288045,-0.100355,0.188287,23.758333,1,sell
3,2022-10-26 01:00:00,24.1500,24.1500,23.7600,23.8400,13772.65,64.054892,83.517327,87.090858,-0.029809,...,-10.369665,-0.020052,-0.052992,0.032940,-0.201703,-0.103966,0.133742,22.383333,1,buy
7,2022-11-06 11:00:00,23.1300,23.6400,22.8200,23.4700,195485.06,33.589815,17.450236,12.459531,0.221789,...,-4.240717,-0.018542,-0.006710,-0.011832,0.000000,0.075655,0.200640,25.909167,1,sell
10,2022-11-08 06:00:00,19.2700,19.5500,17.0400,17.9400,2324262.07,16.426697,21.681996,20.663002,0.096723,...,-10.781879,-0.209272,-0.227469,0.018198,0.000000,-0.016908,0.208816,24.600833,0,sell
27,2022-12-27 07:00:00,0.3229,0.3241,0.3227,0.3232,235279.00,34.957293,35.958102,36.687176,-0.013187,...,5.007458,0.001522,0.000284,0.001238,0.396215,1.688159,0.002535,0.307438,1,sell


(7154, 505)

# Train_test_split by ticker group

In [3]:
from sklearn.model_selection import GroupShuffleSplit 

splitter = GroupShuffleSplit(test_size=0.1, n_splits=2, random_state = 7)

split = splitter.split(df, groups=df['ticker'])
train_inds, test_inds = next(split)

train_df = df.iloc[train_inds]
test_df = df.iloc[test_inds]

# Pytorch

### Create dataset

In [4]:
test_size=0.2

x_data = train_df.drop(['target', 'time', 'ticker', 'pattern', 'ttype'], axis=1)
y_data = train_df['target']
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=test_size, shuffle=True)
input_channels = x_data.shape[1]
output_size = len(set(y_data))

scaler = StandardScaler()
x_train[x_train.columns] = scaler.fit_transform(x_train)
x_valid[x_valid.columns] = scaler.transform(x_valid)

x_train = torch.tensor(x_train.values, dtype=torch.float32)
x_valid = torch.tensor(x_valid.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_valid = torch.tensor(y_valid.values, dtype=torch.float32)

display(type(x_train), type(y_train))

torch.Tensor

torch.Tensor

### Find available device

In [5]:
# find available device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Build model

In [6]:
class SigModel(torch.nn.Module):
    def __init__(self, input_channels):
        super(SigModel, self, ).__init__()
        self.layers = torch.nn.Sequential()
        self.layers.add_module('lin1', torch.nn.Linear(input_channels, 64))
        self.layers.add_module('relu1', torch.nn.ReLU())
        self.layers.add_module('do1', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin2', torch.nn.Linear(64, 128))
        self.layers.add_module('relu2', torch.nn.ReLU())
        self.layers.add_module('do2', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin3', torch.nn.Linear(128, 96))
        self.layers.add_module('relu3', torch.nn.ReLU())
        self.layers.add_module('do3', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin4', torch.nn.Linear(96, 32))
        self.layers.add_module('relu4', torch.nn.ReLU())
        self.layers.add_module('do4', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin5', torch.nn.Linear(32, 1))
        self.layers.add_module('sigmoid', torch.nn.Sigmoid())
    
    def forward(self, input):
        return self.layers(input)

# Train model

In [7]:
from torch import nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

# train function
def train_epoch(model, train_loader, criterion, optimizer, scheduler):
    # put the model in train mode
    model.train()
    
    x_train, x_valid, y_train, y_valid = train_loader

    # get output of the model
    train_preds = model(x_train).squeeze()
    # calculate train loss
    train_loss = criterion(train_preds, y_train)
    train_acc = (train_preds.round() == y_train).float().mean()
    
    # set gradient to zero to prevent it accumulation
    optimizer.zero_grad() # ~ model.zero_grad()
    # calculate gradient
    train_loss.backward() 
    # update weights
    optimizer.step()
    
    # put the model in evaluation mode
    model.eval()

    with torch.no_grad():
        val_preds = model(x_valid).squeeze()
        val_loss = criterion(val_preds, y_valid)
        val_acc = (val_preds.round() == y_valid).float().mean()
    
    # update weights according to gradient value
    scheduler.step(val_loss)
    
    return train_loss, train_acc, val_loss, val_acc

# Initialize model
if CFG.train_NN:
    model = SigModel(input_channels).to(device)

    # Number of epochs
    epochs = 100000

    # Send data to the device
    x_train, x_valid = x_train.to(device), x_valid.to(device)
    y_train, y_valid = y_train.to(device), y_valid.to(device)
    train_loader = x_train, x_valid, y_train, y_valid

    # Empty loss lists to track values
    epoch_count, train_loss_values, valid_loss_values = [], [], []

    criterion = nn.BCELoss()
    learning_rate = 1e-6
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2000, threshold=1e-2)

    # Loop through the data
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc, val_loss, val_acc = train_epoch(model, train_loader, criterion, optimizer, scheduler)

        # Print progress a total of 20 times
        if epoch % int(epochs / 20) == 0:
            print(f'Epoch: {epoch:4.0f} | Train Loss: {train_loss:.5f}, Train Acc: {train_acc:.5f}\
                Validation Loss: {val_loss:.5f}, Val Acc: {val_acc:.5f}\
                    LR: {optimizer.state_dict()["param_groups"][0]["lr"]}')

            epoch_count.append(epoch)
            train_loss_values.append(train_loss.cpu().detach().numpy())
            valid_loss_values.append(val_loss.cpu().detach().numpy())


# Plot NN train results

In [8]:
import matplotlib.pyplot as plt

if CFG.train_NN:
    plt.plot(epoch_count, train_loss_values, label='Training Loss')
    plt.plot(epoch_count, valid_loss_values, label='Validation Loss')
    plt.title('Training & Validation Loss Curves')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

# Select features

In [9]:
palette = ['#302c36', '#037d97', '#E4591E', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']

blk = Style.BRIGHT + Fore.BLACK
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

def lgbm_tuning(df, permut=False, boruta=False):
    features = [c for c in df.columns if c not in ['time', 'target', 'ticker', 'pattern', 'ttype']]
    groups = df['ticker']

    outer_cv_score = [] # store all cv scores of outer loop inference

    perm_df_ = pd.DataFrame()
    feature_importances_ = pd.DataFrame()
    boruta_df_ = pd.DataFrame()
    
    for i in range(CFG.n_repeats):
        print(f'Repeat {blu}#{i+1}')
        
        if task_type == 'cls':
            y_fold = df['target']
            kf = StratifiedGroupKFold(n_splits=CFG.n_folds, shuffle=True, random_state=180820231)
            eval_metric = 'logloss'
        else:
            y_fold = (df['target'] - df['close']) / df['close']
            kf = GroupKFold(n_splits=CFG.n_folds)
            eval_metric = 'mse'

        X, y = df[features], y_fold
        oof = np.zeros(len(df))
        models_ = [] # Used to store models trained in the inner loop.
        
        # Stratify based on Class and Alpha (3 types of conditions)
        for fold, (train_idx, val_idx) in enumerate(kf.split(X, y, groups)):
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[train_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[train_idx]
            y_val = y.iloc[val_idx]

            if task_type == 'cls':
                clf = lgb.LGBMClassifier(**params)
            else:
                clf = lgb.LGBMRegressor(**params)
            clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], 
                    eval_metric=eval_metric, 
                    callbacks=[lgb.log_evaluation(100)])

            models_.append(clf)

            if task_type == 'cls':
                val_preds = clf.predict_proba(X_val)[:,1]
                val_score = log_loss(y_val, val_preds)
            else:
                val_preds = clf.predict(X_val)
                val_score = mean_squared_error(y_val, val_preds, squared=False)
            
            oof[val_idx] = val_preds
            best_iter = clf.best_iteration_

            print(f'Fold: {blu}{fold + 1:>3}{res}| loss: {blu}{val_score:.5f}{res}| Best iteration: {blu}{best_iter:>4}{res}')

            # permutation importance
            if permut:
                perm = PermutationImportance(clf, scoring=None, n_iter=1, 
                                             random_state=42, cv=None, refit=False).fit(X_val, y_val)

                perm_importance_df = pd.DataFrame({'importance': perm.feature_importances_}, 
                                                    index=X_val.columns).sort_index()

                if perm_df_.shape[0] == 0:
                    perm_df_ = perm_importance_df.copy()
                else:
                    perm_df_ += perm_importance_df

            # gboost feature importance
            f_i = pd.DataFrame(sorted(zip(clf.feature_importances_, X.columns), 
                                      reverse=True, key=lambda x: x[1]), 
                                columns=['Value','Feature'])

            if feature_importances_.shape[0] == 0:
                feature_importances_ = f_i.copy()
            else:
                feature_importances_['Value'] += f_i['Value']
                    
            # BORUTA importance
            if boruta:
                model = BoostBoruta(clf, importance_type='shap_importances', train_importance=False)
                try:
                    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], 
                            eval_metric=eval_metric, 
                            callbacks=[lgb.log_evaluation(100)])
                except RuntimeError:
                    continue
                
                boruta_importance_df = pd.DataFrame({'importance': model.ranking_}, 
                                                        index=X_train.columns).sort_index()
                if boruta_df_.shape[0] == 0:
                    boruta_df_ = boruta_importance_df.copy()
                else:
                    boruta_df_ += boruta_importance_df

        if task_type == 'cls':
            outer_cv = log_loss(y, oof)
        else:
            outer_cv = mean_squared_error(y, oof, squared=False)
        
        outer_cv_score.append(outer_cv)

    print(f'{red} Outer Holdout avg score: {res} log_loss: {red}{np.mean(outer_cv_score):.5f}{res}')
    print(f'{"*" * 50}\n')
    
    if permut:
        perm_df_ = perm_df_.sort_values('importance', ascending=False)
        perm_df_ = perm_df_.reset_index().rename({'index': 'Feature'}, axis=1)
        
    if boruta:
        boruta_df_ = boruta_df_.sort_values('importance')
        boruta_df_ = boruta_df_.reset_index().rename({'index': 'Feature'}, axis=1)
                                    
    feature_importances_ = feature_importances_.sort_values('Value', ascending=False).reset_index(drop=True)
    
    return perm_df_, feature_importances_, boruta_df_, np.mean(outer_cv_score)


params = {
          'n_estimators': 2000,
          'learning_rate': 0.02,
        #   'early_stopping_round': 100,
          'max_depth': 10,
          'subsample' : 0.7,
          'colsample_bytree': 0.85,
          'num_leaves': 24,
          'verbosity': -1,
          'importance_type': 'gain',
          'max_bin': 255,
          'reg_alpha': 1e-6,
          'reg_lambda': 1e-8
        }

task_type = 'cls'

if task_type == 'cls':
    params['boosting_type'] = 'dart'
    params['objective'] = 'binary'
else:
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'regression'

if CFG.select_features:
    perm_df_, feature_importances_, boruta_df_, outer_cv_score = lgbm_tuning(df, permut=True, boruta=True)

# Combine importances and save them

In [10]:
if CFG.select_features:
    perm_df_['rank'] = perm_df_['importance'].rank(ascending=False)
    boruta_df_['rank'] = boruta_df_['importance'].rank()
    feature_importances_['rank'] = feature_importances_['Value'].rank(ascending=False)

    fi = pd.concat([perm_df_[['Feature','rank']], boruta_df_[['Feature','rank']], feature_importances_[['Feature','rank']]])
    fi = fi.groupby('Feature')['rank'].sum().sort_values().reset_index()
    fi.to_csv('feature_importance.csv')
else:
    fi = pd.read_csv('feature_importance.csv')

# Train_test split based on ticker groups

In [11]:
def model_train(df, train_df, test_df, features, task_type, how, n_folds, low_bound, high_bound, train_test): 
    X, groups = train_df[features], train_df['ticker']
    X = pd.concat([X, pd.get_dummies(train_df[['pattern', 'ttype']], drop_first=True)], axis=1)
    y = train_df['target']
    
    if train_test == 'fold':
        oof = np.zeros([train_df['target'].shape[0], 1])
        
        kf = StratifiedGroupKFold(n_splits=n_folds, shuffle=True, random_state=180820231)

        oe_enc = OrdinalEncoder()
        groups = oe_enc.fit_transform(groups.values.reshape(-1, 1))

        print(f"Training with {len(features)} features")
        
        if how == 'lreg':
            scaler = StandardScaler()
            X[X.columns] = scaler.fit_transform(X)
        
        for fold, (fit_idx, val_idx) in enumerate(kf.split(X, y, groups)):
            print(f'Fold #{fold + 1}')
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[fit_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[fit_idx]
            y_val = y.iloc[val_idx]
            
            models = list()
            if how == 'lgbm':
                model = lgb.LGBMClassifier(**params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                        eval_metric='logloss', callbacks = [lgb.log_evaluation(100)])
            elif how == 'lreg':
                model = LogisticRegression(C=0.1, max_iter=100000)#, class_weight='balanced')
                model.fit(X_train, y_train)

            val_preds = model.predict_proba(X_val)
            val_score = log_loss(y_val, val_preds)
            prec_score, prec_obj_pct = confident_score(y_val, val_preds[:,1], low_bound, high_bound)
            print(f'Logloss: {val_score}, Confident objects precision: {prec_score}, % of confident objects: {prec_obj_pct}')
            oof[val_idx, 0] = val_preds[:,1]

        return oof, model
    elif train_test == 'full':
        print('Train on full data')
        X, y = df[features], df['target']
        X = pd.concat([X, pd.get_dummies(df[['pattern', 'ttype']], drop_first=True)], axis=1)
        model = lgb.LGBMClassifier(**params)
        model.fit(X, y, eval_set=[(X, y)], eval_metric='logloss', callbacks = [lgb.log_evaluation(100)])
        return np.zeros([df.shape[0], 1]), model
    else:
        # fit model on full dataset and predict on test
        print("Test fold")
        X_test, y_test = test_df[features], test_df['target']
        X_test = pd.concat([X_test, pd.get_dummies(test_df[['pattern', 'ttype']], drop_first=True)], axis=1)
        model = lgb.LGBMClassifier(**params)
        model.fit(X, y, eval_set=[(X_test, y_test)], eval_metric='logloss', callbacks = [lgb.log_evaluation(100)])
        oof_test = np.zeros([test_df['target'].shape[0], 1])
        oof_test[:,0] = model.predict_proba(X_test)[:,1]
        return oof_test, model

def prepare_features(fi, feature_num):
    ''' Get features, sort them by their time appearance and return for using in train and inference'''
    fi = fi['Feature'].head(feature_num)
    feature_dict = defaultdict(list)
    features = list()

    for f in fi:
        period = f.split('_')
        if period[-1].isdigit():
            feature_dict[int(period[-1])].append('_'.join(period[:-2]))
        else:
            feature_dict[0].append(f)

    feature_dict = dict(sorted(feature_dict.items()))
    
    for item in feature_dict.items():
        if item[0] > 0:
            features.extend([i + f'_prev_{item[0]}' for i in item[1]])
        else:
            features.extend([i for i in item[1]])

    feature_dict['features'] = features + ['Pattern_Trend', 'STOCH_RSI']

    return features, feature_dict

def confident_score(y, oof, low_bound, high_bound):
    ''' Consider only high confident objects for accuracy and precision scores calculation;
        object probability must be lower than low_bound or higher than high_bound '''
    pred_conf = np.zeros_like(oof)
    pred_conf[oof > high_bound] = 1
    pred_conf[oof < low_bound] = 0
    # pred_conf_acc = pred_conf[(oof < low_bound) | (oof > high_bound)]
    pred_conf_prec = pred_conf[(oof > high_bound)]
    # y_conf_acc = y.values.reshape(-1,1)[(oof < low_bound) | (oof > high_bound)]
    y_conf_prec = y.values.reshape(-1,1)[(oof > high_bound)]

    return precision_score(y_conf_prec, pred_conf_prec), y_conf_prec.shape[0]/y.shape[0]

train_test = 'fold'
low_bound, high_bound = 0.31, 0.69
feature_num = 160

params = {
    'boosting_type': 'dart',
    'n_estimators': 2200,
    'learning_rate': 0.02,
    #   'early_stopping_round': 50,
    'max_depth': 10,
    'colsample_bytree': 0.7,
    'subsample': 0.85,
    'subsample_freq': 1,
    'num_leaves': 24,
    'verbosity': -1,
    'max_bin': 255,
    'reg_alpha': 1e-6,
    'reg_lambda': 1e-8,
    'objective': 'binary',
    # 'is_unbalance': True,
    # 'class_weight': 'balanced',
    'metric': 'average_precision'
    }

if CFG.train_LGBM:
    fi = pd.read_csv('feature_importance.csv')
    features, feature_dict = prepare_features(fi, feature_num)
    oof, model = model_train(df, train_df, test_df, features, task_type=task_type, how='lgbm', n_folds=5, low_bound=low_bound, high_bound=high_bound, train_test=train_test)

    if train_test == 'fold':
        y = train_df['target']
        oof_val_score = log_loss(y, oof)
        oof_conf_prec_score, oof_conf_obj_pct = confident_score(y, oof, low_bound, high_bound)
        print(f'Total fold Logloss: {oof_val_score}, Total confident objects precision: {oof_conf_prec_score}, Total % of confident objects: {oof_conf_obj_pct}')
    elif train_test == 'test':
        y_test = test_df['target']
        test_val_score = log_loss(y_test, oof)
        test_conf_prec_score, test_conf_obj_pct = confident_score(y_test, oof, low_bound, high_bound)
        print(f'Total test Logloss: {test_val_score}, Total test confident objects precision: {test_conf_prec_score}, Total % of test confident objects: {test_conf_obj_pct}')
    elif train_test == 'full':
        model.booster_.save_model('lgbm.pkl')
        joblib.dump(model, 'lgbm.pkl')
        # save feature dictionary for further inference
        with open(f'features.json', 'w') as f:
            json.dump(feature_dict, f)
    

Training with 160 features
Fold #1
[100]	valid_0's binary_logloss: 0.671201	valid_0's average_precision: 0.634788
[200]	valid_0's binary_logloss: 0.664086	valid_0's average_precision: 0.646405
[300]	valid_0's binary_logloss: 0.661295	valid_0's average_precision: 0.650736
[400]	valid_0's binary_logloss: 0.656768	valid_0's average_precision: 0.659428
[500]	valid_0's binary_logloss: 0.655518	valid_0's average_precision: 0.6614
[600]	valid_0's binary_logloss: 0.656294	valid_0's average_precision: 0.661311
[700]	valid_0's binary_logloss: 0.654626	valid_0's average_precision: 0.659192
[800]	valid_0's binary_logloss: 0.653843	valid_0's average_precision: 0.658755
[900]	valid_0's binary_logloss: 0.651615	valid_0's average_precision: 0.662891
[1000]	valid_0's binary_logloss: 0.650495	valid_0's average_precision: 0.666382
[1100]	valid_0's binary_logloss: 0.651148	valid_0's average_precision: 0.663738
[1200]	valid_0's binary_logloss: 0.650246	valid_0's average_precision: 0.666324
[1300]	valid_0's

Total fold Logloss: 0.654868571882246, Total confident objects precision: 0.7310313493866424, Total % of confident objects: 0.20181551439574547

Total test Logloss: 0.6532393825806957, Total test confident objects precision: 0.7449392712550608, Total % of test confident objects: 0.17846820809248554

# Count predictions according to pattern

In [14]:
train_df['target'].value_counts()

1    3695
0    2694
Name: target, dtype: int64

In [15]:
train_df.loc[:,'oof'] = oof >= high_bound
train_df.groupby('pattern')['oof'].agg(['mean', 'count'])


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,mean,count
pattern,,
MACD,0.364930,4357
STOCH_RSI,0.125984,2032
